### Search by song idea 
- Give know song topic association with the song idea 
- Give songs written with save song idea 
- Give popular metaphors used in song that shown


In [2]:
from datetime import datetime
from elasticsearch import Elasticsearch
from collections import defaultdict
import json 
import pandas as pd 

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\deela\anaconda3\envs\IR_app\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
es = Elasticsearch(hosts="http://localhost:9200")

In [3]:
index = 'song-corpus'

In [4]:
song_idea = 'attraction for beautiful girls, for her kindness and generosity. how to ask out for a date to get know her better.'


In [5]:
song_topics = [
    'love life',
    'motherhood',
    'staring or blooming love',
    'broken love & sorrow',
    'nature',
    'fatherhood'
]

In [6]:
def get_similarity(base, topics):
    sentences = [base]+ topics
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    sentence_embeddings = model.encode(sentences)
    sim_vec = cosine_similarity(
        [sentence_embeddings[0]],
        sentence_embeddings[1:]
    )
    return sim_vec[0]

In [8]:
get_similarity(song_idea, song_topics)

array([0.55791247, 0.46825704, 0.5582744 , 0.17390269, 0.05412486,
       0.04819918], dtype=float32)

In [11]:
# search songs by topic 
topic = 'love life'
resp = es.search(index=index, query={"match": {"topic": topic}})
resp['hits']['hits']

[{'_index': 'song-corpus',
  '_type': '_doc',
  '_id': 'st0Tp4UBTgOk-OfX322j',
  '_score': 1.6435483,
  '_ignored': ['Lyrics in sinhala.keyword', 'Lyrics in english.keyword'],
  '_source': {'id': 1,
   'title': 'Aadaraya sundara varadaki',
   'topic': 'love life',
   'Lyrics in english': 'Aadaraya sundara varadaki\nkisidaa samaavak nam nathi\nmata pennalaa aaley hati\nnumba dunna duka hondatama athi //\n\nInduvara nuvan daaley vatee paagii\nDahaney velii unmaada vuye maa\nvenekeku dihaa balmuata yali\nmata neth nathai kendiruve kavurundo\n\npululura thalaa veenaa bandin davatee\nrathangili bandee ninnaadaye anuraa\nhada svara thalaa kampitha vevii\nmata maa pavaa nathi kaley kavurundo',
   'Lyrics in sinhala': 'ආදරය සුන්දර වරදකි\nකිසිදා සමාවක්\u200c නම් නැති\nමට පෙන්නලා ආලේ හැටි\nනුඹ දුන්නු දුක හොඳටම ඇති//\n\nඉඳුවර නුවන් දෑලේ වැටී පෑහී\nබැහැරී වෙලී උන්මාද වූයේ මා\nවෙනකෙකු දිහා බැලුමට යලිත්\nමට නෙත් නැතැයි කෙඳිරුවේ කවුරුන්දෝ\n\nපුළුලුරුතලා වීණා බඳින් දැවටී\nරතැඟිලි බැඳී නින්නාදයේ අනුරා\

In [13]:
# get all metaphors
hits = resp['hits']['hits']

all_annotation = []
all_data = {"Singer": [], "Musician": [], "Writers": [], "Topic": []}

for h in hits:
    h = h['_source']
    all_annotation.extend(h['Annotation'])
    try:
        all_data['Singer'].append((h['Singer'], h['title']))
    except:
        pass 

    try:
        all_data['Musician'].append((h['Music'], h['title']))
    except:
        pass 

    try:
        all_data['Writers'].append((h['Lyrics'], h['title']))
    except:
        pass 

    try:
        all_data['Topic'].append(h['topic'])
    except:
        pass 


all_annotation

[{'Line': 6,
  'Metaphore': 'nuvan daaley',
  'Object-sinhala': 'nuvan',
  'Object-english': 'eye',
  'Subject-sinhala': 'induwara',
  'Subject-english': 'sun'},
 {'Line': 2,
  'Metaphore': 'adaraya sundara varadaki',
  'Object-sinhala': 'adaraya',
  'Object-english': 'love',
  'Subject-sinhala': 'sundara varadaki',
  'Subject-english': 'mistake'},
 {'Line': 14,
  'Metaphore': 'hada svara',
  'Object-sinhala': 'hada',
  'Object-english': 'heart',
  'Subject-sinhala': 'svara',
  'Subject-english': 'notes'},
 {'Line': 12,
  'Metaphore': 'veenaa bandin',
  'Object-sinhala': 'veena',
  'Object-english': 'harps',
  'Subject-sinhala': 'bandin',
  'Subject-english': 'hip'},
 {'Line': 2,
  'Metaphore': 'Eda labaa ganimu tharamin wiyathak',
  'Object-sinhala': 'wikathak',
  'Object-english': 'yard',
  'Subject-sinhala': 'eda',
  'Subject-english': 'space between'},
 {'Line': 1,
  'Metaphore': 'Bol pini wahena welawe',
  'Object-sinhala': 'bol pini wahena welawe',
  'Object-english': 'misty time

In [1]:
import numpy as np 

In [3]:
arr = np.array([1,2,4,1,2,1,5])
np.unique(arr, return_counts=True)

(array([1, 2, 4, 5]),
 array([0, 1, 2, 0, 1, 0, 3], dtype=int64),
 array([3, 2, 1, 1], dtype=int64))